In [ ]:
# !pip install fake_useragent

In [130]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm_notebook

## Описание задачи

Парсинг сайта Rotten Tomatoes с целью сбора данных (рейтинг, название фильма, описание, жанр, год выпуска) по популярным фильмам в июне 2022 года.

На данном сайте список фильмов подгружается динамически по страницам. Обратимся к первой странице со списком фильмов, и запишем запрос к данной странице.

In [4]:
page_link = 'https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page=1'

In [6]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\ProgramData\Anaconda3\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


<Response [200]>

In [7]:
html = response.content

## Получение ссылок с фильмами

С помощью библиотеки BeautifulSoup распарсим страницу и найдем ссылки к фильмам. 
Ссылки к фильмам находятся в теге div с классом discovery-grids-container.

In [9]:
soup = BeautifulSoup(html, 'html.parser')

In [27]:
film_container = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['discovery-grids-container'])

In [162]:
film_links = film_container.findAll(lambda tag: tag.name == 'a')
film_links = [link.attrs['href'] for link in film_links]
film_links[:5]

['/m/top_gun',
 '/m/fantastic_beasts_the_secrets_of_dumbledore',
 '/m/the_northman',
 '/m/what_is_a_woman',
 '/m/x_2022']

Запишем функцию для преобразования полученных данных из атрибута href в нужные ссылки с фильмами

In [36]:
def getPageLinks(page_number):

    # составляем ссылку на страницу поиска
    page_link = 'http://rottentomatoes.com/browse/movies_at_home/sort:popular?page={}'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # наконец, ищем ссылки на фильмы
    film_container = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['discovery-grids-container'])
    film_links = film_container.findAll(lambda tag: tag.name == 'a')
    film_links = ['http://rottentomatoes.com' + link.attrs['href'] for link in film_links]
    
    return film_links

## Получение информации о фильме
Обратимся к странице с фильмом Top Gun и достанем следующую информацию:
- Название фильма.
- Рейтинг фильма.
- Описание фильма.
- Жанр фильма.
- Год выпуска.

In [45]:
film_page = 'http://rottentomatoes.com/m/top_gun'
response = requests.get(film_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Получим название фильма

In [59]:
film_title = soup.find(lambda tag: tag.name == 'h1' and tag.get('class') == ['scoreboard__title'])
film_title = film_title.text
film_title

'Top Gun'

Рейтинги фильма хранятся в аттрибутах тега score-board. Посмотрим на структура данного тега.

In [67]:
scoreboard = soup.find(lambda tag: tag.name == 'score-board' and tag.get('class') == ['scoreboard'])
scoreboard

<score-board audiencescore="83" audiencestate="upright" class="scoreboard" data-qa="score-panel" rating="PG" skeleton="panel" tomatometerscore="57" tomatometerstate="rotten">
<h1 class="scoreboard__title" data-qa="score-panel-movie-title" slot="title">Top Gun</h1>
<p class="scoreboard__info" slot="info">1986, Action/Adventure, 1h 49m</p>
<a class="scoreboard__link scoreboard__link--tomatometer" data-qa="tomatometer-review-count" href="/m/top_gun/reviews?intcmp=rt-scorecard_tomatometer-reviews" slot="critics-count">74 Reviews</a>
<a class="scoreboard__link scoreboard__link--audience" data-qa="audience-rating-count" href="/m/top_gun/reviews?type=user&amp;intcmp=rt-scorecard_audience-score-reviews" slot="audience-count">250,000+ Ratings</a>
<div id="tomatometer_sponsorship_ad" slot="sponsorship"></div>
</score-board>

Запишем функцию для получения данных по рейтингу фильма.

In [100]:
def getRating(soup, rate):
    try:
        obj = soup.find('score-board', attrs={'class':'scoreboard'})
        obj = obj.get(rate)
    except:
        obj=None
    
    return obj

In [101]:
getRating(soup, 'tomatometerscore')

'57'

Получим описание к фильму.

In [93]:
film_plot = soup.find(lambda tag: tag.name == 'div' and tag.get('id') == 'movieSynopsis').text
film_plot = film_plot.replace('\n', '')
film_plot = film_plot.strip()
film_plot

"The Top Gun Naval Fighter Weapons School is where the best of the best train to refine their elite flying skills. When hotshot fighter pilot Maverick (Tom Cruise) is sent to the school, his reckless attitude and cocky demeanor put him at odds with the other pilots, especially the cool and collected Iceman (Val Kilmer). But Maverick isn't only competing to be the top fighter pilot, he's also fighting for the attention of his beautiful flight instructor, Charlotte Blackwood (Kelly McGillis)."

Получение данных о жанре и годе выпуска.

In [112]:
film_genre = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['meta-value', 'genre']).text
film_genre = film_genre.replace('\n', '')
film_genre = film_genre.replace(' ', '')
film_genre = film_genre.replace(',', ', ')
film_genre

'Action, Adventure'

In [124]:
film_year = soup.find(lambda tag: tag.name == 'time').text
film_year = time.strptime(film_year, '%b %d, %Y').tm_year
film_year

1986

Запишем итоговую функцию для сбора данных по фильму.

In [128]:
def getFilmData(film_page):
    film_data = pd.DataFrame(columns=['Title', 'Genre', 'Year', 'Description', 'Tomato Score', 'Tomato Rate', 'Audience Score', 'Audience Rate'], index=[0])
    
    response = requests.get(film_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    film_title = soup.find(lambda tag: tag.name == 'h1' and tag.get('class') == ['scoreboard__title']).text
    film_data['Title'] = film_title
    
    film_plot = soup.find(lambda tag: tag.name == 'div' and tag.get('id') == 'movieSynopsis').text
    film_plot = film_plot.replace('\n', '')
    film_plot = film_plot.strip()
    film_data['Description'] = film_plot
    
    film_genre = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['meta-value', 'genre']).text
    film_genre = film_genre.replace('\n', '')
    film_genre = film_genre.replace(' ', '')
    film_genre = film_genre.replace(',', ', ')
    film_data['Genre'] = film_genre
    
    film_year = soup.find(lambda tag: tag.name == 'time').text
    film_year = time.strptime(film_year, '%b %d, %Y').tm_year
    film_data['Year'] = film_year
    
    film_data['Tomato Score'] = getRating(soup=soup, rate='tomatometerscore')
    film_data['Tomato Rate'] = getRating(soup=soup, rate='tomatometerstate')
    film_data['Audience Score'] = getRating(soup=soup, rate='audiencescore')
    film_data['Audience Rate'] = getRating(soup=soup, rate='audiencestate')
    
    return film_data

In [129]:
getFilmData('http://rottentomatoes.com/m/top_gun')

,Title,Genre,Year,Description,Tomato Score,Tomato Rate,Audience Score,Audience Rate
0,Top Gun,"Action, Adventure",1986,The Top Gun Naval Fighter Weapons School is wh...,57,rotten,83,upright


## Получение информации по всем фильмам
Соберем информацию по всем фильмам. Для начала создадим пустой датафрейм, в который мы будем добавлять информацию по каждому фильму.

In [151]:
final_df = pd.DataFrame(columns=['Title', 'Genre', 'Year', 'Description', 'Tomato Score', 'Tomato Rate', 'Audience Score', 'Audience Rate'])

In [152]:
film_links = getPageLinks(1000)

Запишем цикл для сбора информации по всем фильмам.

In [159]:
for film_link in tqdm_notebook(film_links):
    try: 
        film_data = getFilmData(film_link)
        final_df = pd.concat((final_df, film_data), axis=0)
        time.sleep(0.3)
    except:
        continue

<ipython-input-159-7327607d5902>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for film_link in tqdm_notebook(film_links):


In [161]:
final_df.head()

,Title,Genre,Year,Description,Tomato Score,Tomato Rate,Audience Score,Audience Rate
0,Top Gun,"Action, Adventure",1986,The Top Gun Naval Fighter Weapons School is wh...,57,rotten,83,upright
0,Fantastic Beasts: The Secrets of Dumbledore,"Adventure, Fantasy",2022,Professor Albus Dumbledore (Jude Law) knows th...,47,rotten,83,upright
0,The Northman,"Action, Adventure, Mystery&thriller",2022,Prince Amleth is on the verge of becoming a ma...,89,certified-fresh,64,upright
0,X,"Horror, Mystery&thriller",2022,A group of actors sets out to make an adult fi...,95,certified-fresh,75,upright
0,Emergency,"Comedy, Drama",2022,Kunle (Donald Elise Watkins) and his best frie...,93,certified-fresh,73,upright
